## Concurrency with Shared Variables

## Race Conditions
There are many reasons a function might not work when called concurrently, including deadlock, livelock, and resource starvation. We don’t have space to discuss all of them, so we’ll focus on the most important one, the race condition.

A race condition is a situation in which the program does not give the correct result for some interleavings of the operations of multiple goroutines. Race conditions are pernicious because they may remain latent in a program and appear infrequently, perhaps only under heavy load or when using certain compilers, platforms, or architectures.

In [1]:
var balance int

func Deposit(amount int) { balance = balance + amount }

func Balance() int { return balance }


In [2]:
balance = 0
Deposit(10)
Balance()

10

In [3]:
// no issues here
balance = 0
for i:=1; i<=1e5; i++ {
    Deposit(1)
}

Balance()

100000

In [5]:
// we have our race condition
// remove the 'go' in front of ConcDeposit to run sequentially (and get 1000)
import "sync"

balance = 0

var wg sync.WaitGroup

func ConcDeposit(amount int, wg *sync.WaitGroup){
    defer wg.Done()
    Deposit(amount)
}

for i:=1; i<=1e5; i++ {
    wg.Add(1)
    go ConcDeposit(1, &wg)
}

wg.Wait()
Balance()

99929

This program contains a particular kind of race condition called a data race. A data race occurs whenever two goroutines access the same variable concurrently and at least one of the accesses is a write. A good rule of thumb is that there is no such thing as a benign data race.

There are several ways to avoid a data race:
*  Only read and don't write. Obviously not going to work if updates are essential.
*  Avoid accessing the variable from multiple goroutines
*  The third way to avoid a data race is to allow many goroutines to access the variable, but only one at a time. This approach is known as mutual exclusion.

Here is approach 2:

In [6]:
var deposits = make(chan int) // send amount to deposit
var balances = make(chan int) // receive balance

func Deposit(amount int) { deposits <- amount }
func Balance() int       { return <-balances }

func teller() {
    var balance int // balance is confined to teller goroutine
    for {
        select {
            case amount := <-deposits:
                balance += amount
            case balances <- balance:
        }
    }
}

func init() {
    go teller() // start the monitor goroutine
}

init()

In [7]:
Balance()

0

In [9]:
Balance()

0

In [10]:
Deposit(10)

In [11]:
Balance()

10

In [12]:
Deposit(5)

In [13]:
Balance()

15

In [14]:
Deposit(-7)

In [15]:
Balance()

8

Finally, under approach 3 with mutual exclusion:

In [17]:
type Cake struct{ state string }

func baker(cooked chan<- *Cake) {
    for {
        cake := new(Cake)
        cake.state = "cooked"
        cooked <- cake // baker never touches this cake again
    }
}

func icer(iced chan<- *Cake, cooked <-chan *Cake){
    for cake := range cooked {
        cake.state = "iced"
        iced <- cake // icer never touches this cake again
    }
}

## Mutual Exclusion: sync.Mutex

In [18]:
var (
    sema = make(chan struct{}, 1) // a binary semaphore guarding balance
    balance int
)

func Deposit(amount int) {
    sema <- struct{}{} // acquire token
    balance = balance + amount
    <-sema // release token
}

func Balance() int {
    sema <- struct{}{} // acquire token
    b := balance
    <-sema // release token
    return b
}

This pattern of mutual exclusion is so useful that it is supported directly by the Mutex type from the sync package.

In [19]:
import "sync"

var (
    mu sync.Mutex // guards balance
    balance int
)

func Deposit(amount int) {
    mu.Lock()
    balance = balance + amount
    mu.Unlock()
}

func Balance() int {
    mu.Lock()
    b := balance
    mu.Unlock()
    return b
}


In [20]:
for i:=1; i<=10000; i++ {
    go Deposit(1)
}

In [22]:
Balance()

10000

By convention, the variables guarded by a mutex are declared immediately after the declaration of the mutex itself. If you deviate from this, be sure to document it.

By deferring a call to Unlock, the critical section implicitly extends to the end of the current function, freeing us from having to remember to insert Unlock calls in one or more places far from the call to Lock. Furthermore, a deferred Unlock will run even if the critical section panics, which may be important in programs that make use of recover.

In [23]:
func Balance() int {
    mu.Lock()
    defer mu.Unlock()
    return balance
}

In [24]:
// NOTE: not atomic!
func Withdraw(amount int) bool {
    Deposit(-amount)
    if Balance() < 0 {
        Deposit(amount)
        return false // insufficient funds
    }
    return true
}

This function eventually gives the correct result, but it has a nasty side effect. When an excessive withdrawal is attempted, the balance transiently dips below zero. This may cause a concurrent withdrawal for a modest sum to be spuriously rejected. The problem is that Withdraw is not atomic: it consists of a sequence of three separate operations, each of which acquires and then releases the mutex lock, but nothing locks the whole sequence.

Ideally, Withdraw should acquire the mutex lock once around the whole operation. However, this attempt won’t work:

In [25]:
// NOTE: incorrect!
func Withdraw(amount int) bool {
    mu.Lock()
    defer mu.Unlock()
    Deposit(-amount)
    if Balance() < 0 {
        Deposit(amount)
        return false // insufficient funds
    }
    return true
}

Deposit tries to acquire the mutex lock a second time by calling mu.Lock(), but because mutex locks are not re-entrant—it’s not possible to lock a mutex that’s already locked—this leads to a deadlock where nothing can proceed, and Withdraw blocks forever.

A common solution is to divide a function such as Deposit into two: an unexported function, deposit, that assumes the lock is already held and does the real work, and an exported function Deposit that acquires the lock before calling deposit.

In [27]:
func Withdraw(amount int) bool {
    mu.Lock()
    defer mu.Unlock()
    deposit(-amount)
    if balance < 0 {
        deposit(amount)
        return false // insufficient funds
    }
    return true
}

func Deposit(amount int) {
    mu.Lock()
    defer mu.Unlock()
    deposit(amount)
}

func Balance() int {
    mu.Lock()
    defer mu.Unlock()
    return balance
}

// This function requires that the lock be held.
func deposit(amount int) { balance += amount }

In [28]:
Balance()

10000

In [29]:
Withdraw(15000)

false

In [30]:
Balance()

10000

In [32]:
Withdraw(9000)

true

In [33]:
Balance()

1000

When you use a mutex, make sure that both it and the variables it guards are not exported, whether they are package-level variables or the fields of a struct.

## Read/Write Mutexes: sync.RWMutex
We need a special kind of lock that allows read-only operations to proceed in parallel with each other, but write operations to have fully exclusive access. This lock is called a multiple readers, single writer lock, and in Go it’s provided by sync.RWMutex.

In [34]:
var mu sync.RWMutex
var balance int

func Balance() int {
    mu.RLock() // readers lock
    defer mu.RUnlock()
    return balance
}


RLock can be used only if there are no writes to shared variables in the critical section. In general, we should not assume that logically read-only functions or methods don’t also update some variables. For example, a method that appears to be a simple accessor might also increment an internal usage counter, or update a cache so that repeat calls are faster. If in doubt, use an exclusive Lock.

It’s only profitable to use an RWMutex when most of the goroutines that acquire the lock are readers, and the lock is under contention, that is, goroutines routinely have to wait to acquire it. An RWMutex requires more complex internal bookkeeping, making it slower than a regular mutex for uncontended locks.

## Memory Synchronization


In [42]:
import (
    "fmt"
    "time"
)

var x, y int

go func() {
    x = 1                   // A1
    fmt.Print("y:", y, " ") // A2
}()

go func() {
    y = 1                   // B1
    fmt.Print("x:", x, " ") // B2
}()

time.Sleep(time.Second)

y:0 x:1 

Since these two goroutines are concurrent and access shared variables without mutual exclusion, there is a data race, so we should not be surprised that the program is not deterministic. We might expect it to print any one of these four results, which correspond to intuitive interleavings of the labeled statements of the program.

y:0 x:1

x:0 y:1

x:1 y:1

y:1 x:1

Within a single goroutine, the effects of each statement are guaranteed to occur in the order of execution; goroutines are sequentially consistent. But in the absence of explicit synchronization using a channel or mutex, there is no guarantee that events are seen in the same order by all goroutines.

Where possible, confine variables to a single goroutine; for all other variables, use mutual exclusion.

## Lazy Initialization: sync.Once

In [49]:
import (
    "fmt"
    "sync"
)

var once sync.Once

func myFunc(){
    fmt.Println("Running myFunc")
}

In [50]:
once.Do(myFunc)

Running myFunc


In [51]:
once.Do(myFunc)  // will not run again

## The Race Detector

Even with the greatest of care, it’s all too easy to make concurrency mistakes. Fortunately, the Go runtime and toolchain are equipped with a sophisticated and easy-to-use dynamic analysis tool, the race detector.

Just add the -race flag to your go build, go run, or go test command. This causes the compiler to build a modified version of your application or test with additional instrumentation that effectively records all accesses to shared variables that occurred during execution, along with the identity of the goroutine that read or wrote the variable.

In [67]:
// hasarace.go
// go run -race hasarace.go

import "fmt"

var x int

func main() {
    for i := 1; i <= 1000; i++ {
        go func() {
            x++
        }()
    }

    fmt.Println(x)
}

main()

665



C:\Users\simon.garisch\Desktop\git\golang_stuff\the_gpl>go run -race hasarace.go

WARNING: DATA RACE
Read at 0x000000618ce8 by goroutine 8:
  main.main.func1()
      C:/Users/simon.garisch/Desktop/git/golang_stuff/the_gpl/hasarace.go:12 +0x41

Previous write at 0x000000618ce8 by goroutine 7:
  main.main.func1()
      C:/Users/simon.garisch/Desktop/git/golang_stuff/the_gpl/hasarace.go:12 +0x5d

Goroutine 8 (running) created at:
  main.main()
      C:/Users/simon.garisch/Desktop/git/golang_stuff/the_gpl/hasarace.go:11 +0x59

Goroutine 7 (finished) created at:
  main.main()
      C:/Users/simon.garisch/Desktop/git/golang_stuff/the_gpl/hasarace.go:11 +0x59

999
Found 1 data race(s)
exit status 66

The race detector reports all data races that were actually executed. However, it can only detect race conditions that occur during a run; it cannot prov that none will ever occur. For best results, make sure that your tests exercise your packages using concurrency. Due to extra bookkeeping, a program built with race detection needs more time and memory to run, but the overhead is tolerable even for many production jobs. For infrequently occurring race conditions, letting the race detector do its job can save hours or days of debugging.

***